使用在线，调用OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

OPENAI_API_KEY = ''

llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY)
llm.invoke("how can langsmith help with testing?")


# 使用本地

安装 ollama, 然后运行
ollama run llama2-chinese

In [23]:
# simple
from langchain_community.llms import Ollama
llm = Ollama(model="llama2-chinese")
llm.invoke("海关注册编码")

'\n根据我所知道的信息，海关注册编码是一个独立存在于海关系统中的代码。这个编码可以被用来确认或验证海关货物、人员的登记、清关等活动。\n\n因此，无法提供具体的海关注册编码信息。建议您联系相关方进行确认和验证。'

复杂示例

In [40]:

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import UnstructuredMarkdownLoader, TextLoader, UnstructuredExcelLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain_community.llms import Ollama


loader = UnstructuredMarkdownLoader("/Users/warriorg/Workspace/Notes/doc/customs/关务.md")
data = loader.load()

loader = TextLoader("/Users/warriorg/Downloads/xxxx.txt")
data2 = loader.load()

# loader = UnstructuredExcelLoader("/Users/warriorg/Downloads/2024研发中心前后端工作.xlsx")
# data3 = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents([data[0], data2[0]])

embeddings = OllamaEmbeddings(model="llama2-chinese")
vector = FAISS.from_documents(documents, embeddings)
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
Question: {input}""")

llm = Ollama(model="llama2-chinese")
document_chain = create_stuff_documents_chain(llm, prompt)

# document_chain.invoke({
#     "input": "叶群",
#     "context": [Document(page_content="朗新一诺顶级开发工程师，代表了苏州软件行业的最高水准")]
# })

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# response = retrieval_chain.invoke({"input": "企业海关注册编码"})
# print(response["answer"])

# response = retrieval_chain.invoke({"input": "什么是内销"})
# print(response["answer"])

response = retrieval_chain.invoke({"input": "以叶群为主人公写一篇爽文"})
print(response["answer"])

KeyboardInterrupt: 

切分文档到向量数据库

In [67]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, UnstructuredMarkdownLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

loader = UnstructuredMarkdownLoader("/Users/warriorg/Workspace/Notes/doc/customs/关务.md")
data = loader.load()

# load the document and split it into chunks
loader = TextLoader("/Users/warriorg/Downloads/xxxx.txt")
data2 = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents([data[0], data2[0]])

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function, persist_directory="/Users/warriorg/Downloads/chroma")
db.persist

# query it
query = "叶群"
docs = db.similarity_search(query)

# print results
print(docs[0])

page_content='叶群是一个长相甜美，性格温柔的老姑娘' metadata={'source': '/Users/warriorg/Downloads/xxxx.txt'}


In [69]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)


embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(embedding_function=embedding_function, persist_directory="/Users/warriorg/Downloads/chroma")

llm = Ollama(model="llama2-chinese")
qa_chain=RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(), chain_type="stuff")
response = qa_chain({"query": "叶群"})
print(response)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)